In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

## Document Loader

In [4]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader("./wine_reviews/winemag-data-130k-v2.csv")
docs = loader.load()
for i, d in enumerate(docs[:2]):
    print(i, d)

0 page_content=': 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia' metadata={'source': './wine_reviews/winemag-data-130k-v2.csv', 'row': 0}
1 page_content=': 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Quinta dos Avidagos 2011 Avidagos Red 

## Pinecone에 Store 실행

In [8]:
# embedding 모델 객체 생성
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model=os.getenv("OPENAI_EMBEDDING_MODEL"))

In [10]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 클라이언트를 초기화(객체 생성)
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

# index 이름
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")

# index 초기화 (객체 생성)   # 인덱스 존재 안하면 생성
existing_indexes = pc.list_indexes()

# 이름만 추출
index_names = [index['name'] for index in existing_indexes.indexes]
# print(index_names)

if PINECONE_INDEX_NAME not in index_names:
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
    print(f"Index '{PINECONE_INDEX_NAME}' created successfully.")

else:
    print(f"Index '{PINECONE_INDEX_NAME}' already exists.")



Index 'wine-reviews' created successfully.


In [11]:
# split 하기
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 (예: 1000자씩 분할)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# 문서 분할
docs = text_splitter.split_documents(docs)

In [13]:
# vector store에 저장
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_documents(
    docs,
    embedding=embeddings,
    index_name = os.getenv("PINECONE_INDEX_NAME"),
    namespace = os.getenv("PINECONE_NAMESPACE")
)

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-embedding-3-small in organization org-vFQ0xTKTHuV2ZtZbSgsF47tK on tokens per min (TPM): Limit 1000000, Used 1000000, Requested 155950. Please try again in 9.357s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

* 업로딩 시간 좀 걸림. 4~50분까지도..